In [1]:
## Question_tester Display LLM generated answers to SME questions for specific hadm_id context data
## based on huggingface example https://huggingface.co/learn/cookbook/en/advanced_rag


In [6]:
## load data from csv
# path for input and target data tables
# path = "/work/AI-EHR/discharge_me/data/test_phase_2/all_text_data_by_hadm_id"

diagnosis_path = '/work/AI-EHR/discharge_me/data/test_phase_2/diagnosis_hadm.csv'
discharge_path = '/work/AI-EHR/discharge_me/data/test_phase_2/discharge.csv'
edstays_path = '/work/AI-EHR/discharge_me/data/test_phase_2/edstays.csv'
radiology_path = '/work/AI-EHR/discharge_me/data/test_phase_2/radiology.csv'
triage_path = '/work/AI-EHR/discharge_me/data/test_phase_2/triage.csv'
target_path = '/work/AI-EHR/discharge_me/data/test_phase_2/discharge_target.csv'
discharge_sections_path = '/work/AI-EHR/discharge_me/data/test_phase_2/discharge_sections.csv'
radiology_sections_path = '/work/AI-EHR/discharge_me/data/test_phase_2/radiology_sections.csv'

import pandas as pd
import re
import os

# read data
diagnosis_df = pd.read_csv(diagnosis_path, keep_default_na=False)
discharge_df = pd.read_csv(discharge_path, keep_default_na=False)
edstays_df = pd.read_csv(edstays_path, keep_default_na=False)
radiology_df = pd.read_csv(radiology_path, keep_default_na=False)
triage_df = pd.read_csv(triage_path, keep_default_na=False)
target_df = pd.read_csv(target_path, keep_default_na=False)

discharge_sections_df = pd.read_csv(discharge_sections_path, keep_default_na=False)
radiology_sections_df = pd.read_csv(radiology_sections_path, keep_default_na=False)

In [7]:
discharge_sections_df.head()

,Unnamed: 0,subject_id,hadm_id,CC,Service,Major Surgical Procedure,HPI,PMH,SOC,FH,...,Problem List,Physical Exam,Medication Lists,Pertinent Results,BHC,Transitional Issues,Disposition,Discharge Instructions,Followup Instructions,Discharge Diagnosis
0,0,10001884,24962904,Shortness of Breath,"MEDICINE\n \nAllergies: \nIV Dye, Iodine Conta...",N/A,Ms. ___ is a ___ female with history of \nCOPD...,- COPD/Asthma on home 2L O2\n- Atypical Chest ...,___,Mother with asthma and hypertension. Father wi...,...,,ADMISSION PHYSICAL EXAM:\n====================...,is accurate and complete.\n1. Acetaminophen 32...,ADMISSION LABS: \n=========================\n_...,Ms. ___ is a ___ female with history of \nCOPD...,==========================\n[] For pt's contin...,Extended Care\n \nFacility:\n___\n \n___ Diagn...,"Dear Ms. ___,\n\nYou were admitted to ___ afte...",,PRIMARY:\nCOPD Exacerbation\n\nSECONDARY:\nAfi...
1,1,10003019,22774359,fever,MEDICINE\n \nAllergies: \nRagweed / morphine /...,none,Mr ___ is a ___ with h/o stage IV Hodgkins c1d...,"1. Sarcoidosis, dx skin bx: intestinal & pulmo...",___,"Mother: ___, cardiac disease. \nFather: diver...",...,,ADMISSION EXAM\nVitals: 124/67 on neosynephrin...,The Preadmission Medication list is accurate a...,ADMISSION LABS\n___ 10:40AM BLOOD WBC-0.2* RBC...,___ male with h/o Hodgkin's lymphoma C1D17 ABV...,,Home With Service\n \nFacility:\n___\n \nDisch...,"Dear Mr. ___,\n\nIt has been our pleasure to b...",,"Primary Diagnosis\nNeutropenic Fever, no sourc..."
2,2,10003299,29323205,"left leg weakness, falls",NEUROLOGY\n \nAllergies: \nIodine-Iodine Conta...,None,___ is a ___ RH female with a PMHx of\nparamed...,- prior paramedian pontine infarct (___) \n- r...,___,Mother had stroke in her ___ or ___. Her pate...,...,,Admission Exam:\nVitals: T: 97.4 P: 65 R: 16 ...,The Preadmission Medication list is accurate a...,___ 01:10PM GLUCOSE-125* UREA N-9 CREAT-0.9 ...,___ RH female with a PMHx of paramedian pontin...,,Home With Service\n \nFacility:\n___\n \nDisch...,Dear ___ were hospitalized due to symptoms of ...,,Ischemic stroke
3,3,10003502,20459702,Nausea,MEDICINE\n \nAllergies: \nnifedipine / Amitrip...,None,Ms. ___ is an ___ with atrial fibrillation/flu...,Hypertension/hyperlipidemia\nCoronary artery d...,___,Mother deceased at ___ yo from breast cancer. ...,...,,On admission: \nVS 97.4 140/P 62 18 96\nGEN Al...,The Preadmission Medication list is accurate a...,___ 10:15AM BLOOD WBC-6.4 RBC-3.64* Hgb-11.1* ...,Hospitalization Summary: \nMs. ___ is an ___ y...,- monitoring of volume status and titration of...,Home With Service\n \nFacility:\n___\n \nDisch...,It was a pleasure caring for you at ___ \n___....,,Primary: \nAcute diastolic CHF exacerbation\nN...
4,4,10004322,28755331,multiple falls at group home,MEDICINE\n \nAllergies: \nNo Known Allergies /...,None,Mr. ___ is a ___ year old male with a h/o psyc...,Psychosis \nDiabetes \nCOPD,___,Unknown to patient.,...,,ADMISSION PHYSICAL EXAM: \nVS: 98.1 117/62 97 ...,The Preadmission Medication list is accurate a...,ADMISSION LABS:\n___ 03:40PM BLOOD WBC-17.7* R...,___ with h/o psychosis admitted because of mul...,-He was discharged back home to the ___ Home\n...,Home With Service\n \nFacility:\n___\n \nDisch...,"Dear Mr. ___,\n\nIt was our pleasure to care f...",,Primary: mechanical fall\nSecondary: \npsychos...


In [ ]:
# questions = [
# "Given the History of Present Illness, What is the relevant past medical history?",
# "Given the HPI, chief complaint, and initial ICD diagnosis, Why was the patient admitted to the hospital? What symptoms, medical condition, or other reason caused the patient to come to or be brought into the hospital?",
# "Based on the admission medication list, what other existing or ongoing medical conditions did the patient have?",
# "Given the history of present illness, admission and discharge diagnosis, What medical conditions or symptoms was this patient treated for in the hospital?",
# "given the history of present illness, What was the initial treatment course?",
# "Given the History of Present illness, What was the initial diagnostic work up and pertinent results?",
# "Given the radiology notes, What radiology, imaging or other studies that were performed and for what inducation? Summarize the impressions from each study. Make a table of Imaging study, indication, and impression."
# ]

hadm_id = 20154746
df = discharge_sections_df
cur_df = df[df['hadm_id'] == hadm_id]
    

##    
questions[0] = "Given the Chief Complaint and History of Present Illness, What is the patient's relevant past medical history?"

context_str = ''    
cols = ['CC']
context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
context_str = f"The chief complaint was: {context_data} \n"
cols = ['HPI']
context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
context_str = context_str + f"The History of present illness:  \n{context_data} \n"  
context[0] = context_str

##
question[1] = "Given the HPI, chief complaint, and initial ICD diagnosis, Why was the patient admitted to the hospital? What symptoms, medical condition, or other reason caused the patient to come to or be brought into the hospital?"
context_str = ''    
cols = ['CC']
context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
context_str = f"The chief complaint was: {context_data} \n"
cols = ['HPI']
context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
context_str = context_str + f"The History of present illness:  \n{context_data} \n"  
df = diagnosis_df
cols = ['icd_title']
cur_df = df[df['hadm_id'] == hadm_id]
context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n ')
context_str = context_str + f"The patient was initially diagnosed with ICD Diagnosis titles --- \n{context_data}\n "
    
context[1] = context_str


    # # cols = ['Medication Lists']
    # # context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    # # context_str = context_str + f"The admission and discharge medication lists:\n{context_data} \n"
    # # cols = ['Pertinent Results']
    # # context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    # # context_str = context_str + f"The Pertinent Results were:\n{context_data} \n"
    # cols = ['Major Surgical Procedure']
    # context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    # context_str = context_str + f"The major procedures were --- \n{context_data} \n"
    # cols = ['Discharge Diagnosis']
    # context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    # context_str = context_str + f"At discharge, the patient was diagnosed with --- \n{context_data} \n"
    # context_str = context_str.replace(":", '---').replace("==", '')


In [ ]:
prompt_in_chat_format = [
    {
        "role": "system",
        "content": """You are an internal medicine physician conducting a review of a patient's EHR. 
        You need to find relevant and accurate information contained in the text of clinical notes. 
        Using the information contained in the context provided, answer the question about the patient's medical records..
Respond only to the question asked, response should be concise and relevant to the question.
If the answer cannot be deduced from the context, respond that the information can not be deduced from the information provided.""",
    },
    {
        "role": "user",
        "content": """Context:
{context}
---
Please answer the following question.

Question: {question}""",
    },
]
RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
    prompt_in_chat_format, tokenize=False, add_generation_prompt=True
)
print(RAG_PROMPT_TEMPLATE)

In [8]:
# segment discharge notes

def extract_text_by_subheading_discharge(text):
    
    headings_regex = {
        'CC': r'(CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:)(.*?)(?=Service:|Specialty:|Unit:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|Major Surgical or Invasive Procedure:|HPI:|History of Present Illness:|PMH:|Past Medical History:|Social History:|SOC:|SH:|Family History:|PFH:|FH:|Past Surgical History:|PSH:|Problem List:|Problems:|Physical Exam:|PE:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|BHC:|Brief Hospital Course:|Disposition:|Dispo:|Discharge Instructions:|Patient Instructions:|Followup Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
        'Service': r'(Service:|Specialty:|Unit:)(.*?)(?=CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:|Major Surgical or Invasive Procedure:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|HPI:|History of Present Illness:|PMH:|Past Medical History:|Social History:|SOC:|SH:|Family History:|PFH:|FH:|Past Surgical History:|PSH:|Problem List:|Problems:|Physical Exam:|PE:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|BHC:|Brief Hospital Course:|Disposition:|Dispo:|Discharge Instructions:|Patient Instructions:|Followup Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
        'Major Surgical Procedure': r'(Major Surgical or Invasive Procedure:)(.*?)(?=CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:|Service:|Specialty:|Unit:|HPI:|History of Present Illness:|PMH:|Past Medical History:|Social History:|SOC:|SH:|Family History:|PFH:|FH:|Past Surgical History:|PSH:|Problem List:|Problems:|Physical Exam:|PE:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|BHC:|Brief Hospital Course:|Disposition:|Dispo:|Discharge Instructions:|Patient Instructions:|Followup Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
        'HPI': r'(HPI:|History of Present Illness:)(.*?)(?=CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:|Service:|Specialty:|Unit:|Major Surgical or Invasive Procedure:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|PMH:|Past Medical History:|Social History:|SOC:|SH:|Family History:|PFH:|FH:|Past Surgical History:|PSH:|Problem List:|Problems:|Physical Exam:|PE:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|BHC:|Brief Hospital Course:|Disposition:|Dispo:|Discharge Instructions:|Patient Instructions:|Followup Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
        'PMH': r'(PMH:|Past Medical History:)(.*?)(?=CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:|Service:|Specialty:|Unit:|Major Surgical or Invasive Procedure:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|HPI:|History of Present Illness:|Social History:|SOC:|SH:|Family History:|PFH:|FH:|Past Surgical History:|PSH:|Problem List:|Problems:|Physical Exam:|PE:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|BHC:|Brief Hospital Course:|Disposition:|Dispo:|Discharge Instructions:|Patient Instructions:|Followup Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
        'SOC': r'(Social History:|SOC:|SH:)(.*?)(?=CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:|Service:|Specialty:|Unit:|Major Surgical or Invasive Procedure:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|HPI:|History of Present Illness:|PMH:|Past Medical History:|Family History:|PFH:|FH:|Past Surgical History:|PSH:|Problem List:|Problems:|Physical Exam:|PE:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|BHC:|Brief Hospital Course:|Disposition:|Dispo:|Discharge Instructions:|Patient Instructions:|Followup Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
        'FH': r'(Family History:|PFH:|FH:)(.*?)(?=CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:|Service:|Specialty:|Unit:|Major Surgical or Invasive Procedure:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|HPI:|History of Present Illness:|PMH:|Past Medical History:|Social History:|SOC:|SH:|Past Surgical History:|PSH:|Problem List:|Problems:|Physical Exam:|PE:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|BHC:|Brief Hospital Course:|Disposition:|Dispo:|Discharge Instructions:|Patient Instructions:|Followup Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
        'Past Surgical History': r'(Past Surgical History:|PSH:)(.*?)(?=CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:|Service:|Specialty:|Unit:|Major Surgical or Invasive Procedure:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|HPI:|History of Present Illness:|PMH:|Past Medical History:|Social History:|SOC:|SH:|Family History:|PFH:|FH:|Problem List:|Problems:|Physical Exam:|PE:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|BHC:|Brief Hospital Course:|Disposition:|Dispo:|Discharge Instructions:|Patient Instructions:|Followup Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
        'Problem List': r'(Problem List:|Problems:)(.*?)(?=CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:|Service:|Specialty:|Unit:|Major Surgical or Invasive Procedure:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|HPI:|History of Present Illness:|PMH:|Past Medical History:|Social History:|SOC:|SH:|Family History:|PFH:|FH:|Past Surgical History:|PSH:|Physical Exam:|PE:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|BHC:|Brief Hospital Course:|Disposition:|Dispo:|Discharge Instructions:|Patient Instructions:|Followup Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
        'Physical Exam': r'(Physical Exam:|PE:)(.*?)(?=CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:|Service:|Specialty:|Unit:|Major Surgical or Invasive Procedure:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|HPI:|History of Present Illness:|PMH:|Past Medical History:|Social History:|SOC:|SH:|Family History:|PFH:|FH:|Past Surgical History:|PSH:|Problem List:|Problems:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|BHC:|Brief Hospital Course:|Disposition:|Dispo:|Discharge Instructions:|Patient Instructions:|Followup Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
        'Medication Lists': r'(Medication lists:|Admission Medications:|Medications on Admission:|Preadmission Medication list)(.*?)(?=CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:|Service:|Specialty:|Unit:|Major Surgical or Invasive Procedure:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|HPI:|History of Present Illness:|PMH:|Past Medical History:|Social History:|SOC:|SH:|Family History:|PFH:|FH:|Past Surgical History:|PSH:|Problem List:|Problems:|Physical Exam:|PE:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|BHC:|Brief Hospital Course:|Disposition:|Dispo:|Discharge Instructions:|Patient Instructions:|Followup Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
        'Pertinent Results': r'(Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:)(.*?)(?=CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:|Service:|Specialty:|Unit:|Major Surgical or Invasive Procedure:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|HPI:|History of Present Illness:|PMH:|Past Medical History:|Social History:|SOC:|SH:|Family History:|PFH:|FH:|Past Surgical History:|PSH:|Problem List:|Problems:|Physical Exam:|PE:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|BHC:|Brief Hospital Course:|Disposition:|Dispo:|Discharge Instructions:|Patient Instructions:|Followup Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
        'BHC': r'(BHC:|Brief Hospital Course:|Hospital Course)(.*?)(?=CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:|Service:|Specialty:|Unit:|Major Surgical or Invasive Procedure:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|HPI:|History of Present Illness:|PMH:|Past Medical History:|Social History:|SOC:|SH:|Family History:|PFH:|FH:|Past Surgical History:|PSH:|Problem List:|Problems:|Physical Exam:|PE:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|Disposition:|Dispo:|Discharge Instructions:|Patient Instructions:|Followup Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
        'Transitional Issues': r'(Transitional Issues:|Transitional Issues|TRANSITIONAL ISSUES:)(.*?)(?=CODE STATUS:|Disposition:|Dispo:|CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:|Service:|Specialty:|Unit:|Major Surgical or Invasive Procedure:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|HPI:|History of Present Illness:|PMH:|Past Medical History:|Social History:|SOC:|SH:|Family History:|PFH:|FH:|Past Surgical History:|PSH:|Problem List:|Problems:|Physical Exam:|PE:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|BHC:|Brief Hospital Course:|Discharge Instructions:|Patient Instructions:|Followup Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
        'Disposition': r'(Disposition:|Dispo:)(.*?)(?=CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:|Service:|Specialty:|Unit:|Major Surgical or Invasive Procedure:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|HPI:|History of Present Illness:|PMH:|Past Medical History:|Social History:|SOC:|SH:|Family History:|PFH:|FH:|Past Surgical History:|PSH:|Problem List:|Problems:|Physical Exam:|PE:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|BHC:|Brief Hospital Course:|Discharge Instructions:|Patient Instructions:|Followup Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
        'Discharge Instructions': r'(Discharge Instructions:|Patient Instructions:)(.*?)(?=CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:|Service:|Specialty:|Unit:|Major Surgical or Invasive Procedure:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|HPI:|History of Present Illness:|PMH:|Past Medical History:|Social History:|SOC:|SH:|Family History:|PFH:|FH:|Past Surgical History:|PSH:|Problem List:|Problems:|Physical Exam:|PE:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|BHC:|Brief Hospital Course:|Disposition:|Dispo:|Followup Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
        'Followup Instructions': r'(Followup Instructions:)(.*?)(?=CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:|Service:|Specialty:|Unit:|Major Surgical or Invasive Procedure:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|HPI:|History of Present Illness:|PMH:|Past Medical History:|Social History:|SOC:|SH:|Family History:|PFH:|FH:|Past Surgical History:|PSH:|Problem List:|Problems:|Physical Exam:|PE:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|BHC:|Brief Hospital Course:|Disposition:|Dispo:|Discharge Instructions:|Patient Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
        'Discharge Diagnosis': r'(Discharge Diagnosis:|Diagnosis:)(.*?)(?=Discharge Condition:|CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:| Service:|Specialty:|Unit:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|HPI:|History of Present Illness:|PMH:|Past Medical History:|Social History:|SOC:|SH:|Family History:|PFH:|FH:|Past Surgical History:|PSH:|Problem List:|Problems:|Physical Exam:|PE:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|BHC:|Brief Hospital Course:|Disposition:|Dispo:|Discharge Instructions:|Patient Instructions:|Followup Instructions:|Transitional Issues:$)',
    }

    extracted_text = {}
    for heading, regex in headings_regex.items():
        match = re.search(regex, text, re.DOTALL)
        if match:
            extracted_text[heading] = match.group(0).strip()
        else:
            extracted_text[heading] = None        

    return extracted_text

#apply function
discharge_sections_df = discharge_df['text'].apply(extract_text_by_subheading_discharge).apply(pd.Series)

# Concatenating the columns from radiology_df into radiology_sections_df
discharge_sections_df= pd.concat([discharge_df[['subject_id', 'hadm_id']], discharge_sections_df], axis=1)



In [23]:
# segment radiology notes
def extract_text_by_subheading_radiology(text):
    headings_regex = {
        'EXAMINATION': r'(?:EXAMINATION:|EXAM:!STUDY:)\s*(.*?)(?=\n[A-Za-z ]+:|$)',
        'INDICATION':  r'(?:INDICATION:|CLINICAL INFORMATION:|HISTORY:)\s*(.*?)(?=\n[A-Za-z ]+:|$)',
        'TECHNIQUE': r'(?:TECHNIQUE:|PROCEDURE:)\s*(.*?)(?=\n[A-Za-z ]+:|$)',
        'COMPARISON': r'(?:COMPARISON)\s*(.*?)(?=\n[A-Za-z ]+:|$)',
        'FINDINGS': r'(?:FINDINGS:)\s*(.*?)(?=\n[A-Za-z ]+:|$)',
        'IMPRESSION': r'(^IMPRESSION:|IMPRESSION|RECOMMENDATION)\s*(.*?)$'
    }

    extracted_text = {}

    # Iterate over each subheading and extract corresponding text
    for heading, regex in headings_regex.items():
        match = re.search(regex, text, re.DOTALL)
        if match:
            extracted_text[heading] = match.group(0).strip()
        else:
            extracted_text[heading] = None

    return extracted_text

#apply function
radiology_sections_df = radiology_df['text'].apply(extract_text_by_subheading_radiology).apply(pd.Series)

# Concatenating the columns from radiology_df into radiology_sections_df
radiology_sections_df = pd.concat([radiology_df[['subject_id', 'hadm_id']], radiology_sections_df], axis=1)

In [25]:
import os

data_path = '/work/AI-EHR/discharge_me/data/test_phase_2/'
filename = 'discharge_sections.csv'
full_path = os.path.join(data_path, filename)
discharge_sections_df.to_csv(full_path)


filename = 'radiology_sections.csv'
full_path = os.path.join(data_path, filename)
radiology_sections_df.to_csv('radiology_sections.csv')

In [24]:
radiology_sections_df.head()

,subject_id,hadm_id,EXAMINATION,INDICATION,TECHNIQUE,COMPARISON,FINDINGS,IMPRESSION
0,10001884,24962904,EXAMINATION:\nChest: Frontal and lateral views,INDICATION: History: ___ with dyspnea // eva...,TECHNIQUE: Chest: Frontal and Lateral,COMPARISON: ___,FINDINGS: \n\nMild basilar atelectasis is seen...,IMPRESSION: \n\nMild basilar atelectasis witho...
1,10003019,22774359,None,"CLINICAL INFORMATION: Fever, on chemotherapy....",None,None,FINDINGS: Frontal and lateral views of the ch...,None
2,10003019,22774359,None,"CLINICAL INFORMATION: Neutropenic fever, sept...",None,None,FINDINGS: There has been interval placement o...,IMPRESSION: Interval placement of right inter...
3,10003019,22774359,None,"HISTORY: ___ man with Hodgkin's lymphoma, now...",None,None,None,IMPRESSION: AP chest compared to ___:\n\nMult...
4,10003019,22774359,EXAMINATION: Evaluation of the patient with H...,None,TECHNIQUE: MDCT of the chest was obtained fro...,COMPARISON: ___ PET CT and chest radiograph f...,FINDINGS:\n\nAorta and pulmonary arteries are ...,IMPRESSION:\n\n1. No evidence of new pulmonar...


In [2]:
# !pip install torch transformers transformers accelerate bitsandbytes langchain sentence-transformers faiss-gpu openpyxl pacmap datasets langchain-community ragatouille


    ERROR: Command errored out with exit status 1:
     command: /shared/centos7/anaconda3/2022.05/bin/python -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-pjxralhx/datasets_b5186b1211ba49f8b511b0eecddc6cf7/setup.py'"'"'; __file__='"'"'/tmp/pip-install-pjxralhx/datasets_b5186b1211ba49f8b511b0eecddc6cf7/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /tmp/pip-pip-egg-info-2p2gpy1w
         cwd: /tmp/pip-install-pjxralhx/datasets_b5186b1211ba49f8b511b0eecddc6cf7/
    Complete output (7 lines):
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/tmp/pip-install-pjxralhx/datasets_b5186b1211ba49f8b511b0eecddc6cf7/setup.py", line 10, in <module>
        with open(path.join(

In [ ]:
%reload_ext dotenv
%dotenv


In [ ]:
from tqdm.notebook import tqdm
import pandas as pd
from typing import Optional, List, Tuple
from datasets import Dataset
import matplotlib.pyplot as plt

pd.set_option("display.max_colwidth", None)  # This will be helpful when visualizing retriever outputs

In [ ]:
import datasets

ds = datasets.load_dataset("m-ric/huggingface_doc", split="train")

In [ ]:
from langchain.docstore.document import Document as LangchainDocument

RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]}) for doc in tqdm(ds)
]